# Twitter Scraper / API

In [96]:
# Imports here
import twitter
import json
import tweepy
import geopandas as gpd
import re # regex






In [2]:
# Kuo

import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

#Variables that contains the user credentials to access Twitter API 
access_token = "952108626780422144-DRJswwytxVa2Kqcyn2paSJgITaCCy91"
access_token_secret = "wlGgfyS62lpeH00imJ3dnKc9gzubC3aOAUPvXWr9ac1g3"
consumer_key = "B9199qVuNEMat6QFlyJFiYEFO"
consumer_secret = "n3Uhg5sw5wCcfcKZd5tmjraGdb6AytrudqkHKfgH12nvoY31Ll"

#Variables that contains the user credentials to access Twitter API 
access_token = "952108626780422144-DRJswwytxVa2Kqcyn2paSJgITaCCy91"
access_token_secret = "wlGgfyS62lpeH00imJ3dnKc9gzubC3aOAUPvXWr9ac1g3"
consumer_key = "B9199qVuNEMat6QFlyJFiYEFO"
consumer_secret = "n3Uhg5sw5wCcfcKZd5tmjraGdb6AytrudqkHKfgH12nvoY31Ll"


text_file = open("Output.txt", "w")
#This is a basic listener that just prints received tweets to stdout.
class StdOutListener(StreamListener):

    def on_data(self, data):
        print(data)
        text_file.write(data)
        return True

    def on_error(self, status):
        print(status)

if __name__ == '__main__':

    #This handles Twitter authetification and the connection to Twitter Streaming API
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    stream = Stream(auth, l)

    # This line filter Twitter Streams to capture data by the keywords and bounding location
    stream.filter(track=['#nra'], locations=[-123, 32,-63, 48])





In [71]:
# Kuo
import json
import pandas as pd
import matplotlib.pyplot as plt

tweets_data_path = 'output.txt'

tweets_data = []
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [95]:
# Yulong

consumer_key = 'xcW3f75Y6xEcoKnCrGDrNXSLG'
consumer_secret = 'SNQSDZeDlPoUSa443ElufrWxuCqdljfq6FME7Jin9DZdUQRTKa'
access_token = '153588498-8MoqCHZJG7jrLh8KToTErFAzpOzYMvZkhq1YzYmP'
access_token_secret = 'kw4SF72mY4bv5IuCDQxXQzdkfJMJ7v0t06F2OFMmwMfWT'

"""
api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=consumer_secret,
                  access_token=access_token_key,
                  access_token_secret=access_token_secret)

results = api.GetSearch(
    raw_query='q=%23nra')

print type(results[0])
"""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)


def getTweets(hashtag, count):
    hashtag = '#' + hashtag
    tweets = []
    counter = 0
    for tweet in tweepy.Cursor(api.search, q=hashtag, count=count,
                               lang='en').items():
        if (counter > count):
            print 'Goes over the count'
            break;
        counter += 1
        tweets.append(tweet)
    return tweets

tweets = getTweets('nra', 50)


Goes over the count
                       Name           Handle  \
0                     j.nag    Jennifereally   
1               John Hester  JohnHes32617422   
2               Chris Adams    ChrisCole0526   
3             Carina Pierce     carinapierce   
4        PLAYGROUND DETROIT      PLAYDETROIT   
5            Chloé Caroline  ImChloeCaroline   
6                      None             None   
7          Callipygian Kate      spiffykates   
8             Maddie Conroy    maddiejconroy   
9                      None             None   
10      TMJ-BNA Skl. Trades   tmj_BNA_skltrd   
11                     None             None   
12             Sean Hammond        sean7one7   
13                    Nolan     nolan_smith3   
14    Visionary Thot Leader    GoodTimeHaver   
15               Phil Meyer        philmeyer   
16             Ryan Bradley       RealDMBFan   
17                    Kwame      Kwammentary   
18                     None             None   
19            Design

"\n# tweets[0]\n# print json.dumps(tweets[0]._json, sort_keys=True, indent=2)\nt = tweets[0]\nprint t.user.name + ' @' + t.user.screen_name # twitter handle\nprint t.user.location\nprint t.user.description\nprint t.created_at\nprint t.text\nprint t.retweet_count\nprint t.favorite_count\nprint t.coordinates\nprint t.place\n"

In [105]:
pattern = re.compile('(^[\w]+,\s\w{2}$)') # City, State

def create_gdf(tweets):
    main_list = []
    for t in tweets:
        row = []
        if (t['place']):
            if re.match(pattern, t['place']['full_name']):
                row.append(t['user']['name'])
                row.append(t['user']['screen_name'])
                row.append(t['user']['description'])
                row.append(t['user']['location'])
                # row.append(t['created_at'])
                row.append(t['text'])
                row.append(t['retweet_count'])
                row.append(t['favorite_count'])
                row.append(t['coordinates'])
                row.append(t['place']['full_name'])
                main_list.append(row)
    gdf = gpd.GeoDataFrame(main_list)
    return gdf

gdf = create_gdf(tweets_data)
col_names = ['Name', 'Handle', 'Bio', 'Location', 'Content', 'Retweets', 'Favorites', 'Coordinates', 'Place']
gdf.columns = col_names

print gdf

# print len(tweets)

"""
# tweets[0]
# print json.dumps(tweets[0]._json, sort_keys=True, indent=2)
t = tweets[0]
print t.user.name + ' @' + t.user.screen_name # twitter handle
print t.user.location
print t.user.description
print t.created_at
print t.text
print t.retweet_count
print t.favorite_count
print t.coordinates
print t.place
"""

                       Name           Handle  \
0                     j.nag    Jennifereally   
1               John Hester  JohnHes32617422   
2               Chris Adams    ChrisCole0526   
3             Carina Pierce     carinapierce   
4        PLAYGROUND DETROIT      PLAYDETROIT   
5            Chloé Caroline  ImChloeCaroline   
6          Callipygian Kate      spiffykates   
7             Maddie Conroy    maddiejconroy   
8       TMJ-BNA Skl. Trades   tmj_BNA_skltrd   
9              Sean Hammond        sean7one7   
10                    Nolan     nolan_smith3   
11    Visionary Thot Leader    GoodTimeHaver   
12               Phil Meyer        philmeyer   
13             Ryan Bradley       RealDMBFan   
14                    Kwame      Kwammentary   
15            Design & Mine    designandmine   
16            Marion Marino   marino6_marino   
17            Nate Millheim          oaknate   
18            CheckOutStore  checkoutstoretm   
19                 M. 🐝🌻✨   marinacoates

"\n# tweets[0]\n# print json.dumps(tweets[0]._json, sort_keys=True, indent=2)\nt = tweets[0]\nprint t.user.name + ' @' + t.user.screen_name # twitter handle\nprint t.user.location\nprint t.user.description\nprint t.created_at\nprint t.text\nprint t.retweet_count\nprint t.favorite_count\nprint t.coordinates\nprint t.place\n"